In [1]:
import pandas as pd

imd_df = pd.read_csv('ID 2019 for London.csv')
pop_df = pd.read_csv('population_mid-2021.csv')
lookup_df = pd.read_csv('LSOA_(2011)_to_LSOA_(2021)_to_Local_Authority_District_(2022)_Exact_Fit_Lookup_for_EW_(V3).csv')

In [2]:
# Keep required fields
imd_df = imd_df[['LSOA code (2011)', 'Index of Multiple Deprivation (IMD) Score']]

# Merge and convert 2011 codes to 2021 codes
imd_df = imd_df.merge(lookup_df[['LSOA11CD', 'LSOA21CD']], 
                      left_on='LSOA code (2011)', right_on='LSOA11CD', how='left')

# Keep only 2021 codes and IMD scores
imd_df = imd_df[['LSOA21CD', 'Index of Multiple Deprivation (IMD) Score']]

In [3]:
imd_df

,LSOA21CD,Index of Multiple Deprivation (IMD) Score
0,E01000001,6.2
1,E01000002,5.1
2,E01000003,19.4
3,E01000005,28.7
4,E01032739,13.6
...,...,...
5011,E01033604,33.5
5012,E01033605,40.5
5013,E01035717,14.0
5014,E01035717,22.2


In [4]:
# Convert strings to numeric values and remove thousand separators
pop_df['Total'] = pop_df['Total'].str.replace(',', '').astype(int)
pop_df['F65 and over'] = pop_df['F65 and over'].astype(int)
pop_df['M65 and over'] = pop_df['M65 and over'].astype(int)

# Calculate total population aged 65+ and its proportion
pop_df['pop_65plus'] = (pop_df['F65 and over'] + pop_df['M65 and over']) / pop_df['Total']

In [5]:
pop_df['pop_65plus']

0        0.169364
1        0.162970
2        0.128013
3        0.201863
4        0.146193
           ...   
35667    0.253700
35668    0.227210
35669    0.156352
35670    0.188615
35671    0.180496
Name: pop_65plus, Length: 35672, dtype: float64

In [6]:
# Merge by LSOA21CD
final_df = imd_df.merge(pop_df[['LSOA 2021 Code', 'pop_65plus']], 
                        left_on='LSOA21CD', right_on='LSOA 2021 Code', how='inner')

# Group data by LSOA21CD
final_df = final_df[['LSOA21CD', 'Index of Multiple Deprivation (IMD) Score', 'pop_65plus']]


In [7]:
final_df

,LSOA21CD,Index of Multiple Deprivation (IMD) Score,pop_65plus
0,E01000001,6.2,0.260013
1,E01000002,5.1,0.208244
2,E01000003,19.4,0.197516
3,E01000005,28.7,0.085145
4,E01032739,13.6,0.025107
...,...,...,...
5011,E01033604,33.5,0.083893
5012,E01033605,40.5,0.120337
5013,E01035717,14.0,0.114616
5014,E01035717,22.2,0.114616


In [ ]:
final_df.to_csv('IMD_65plus_merged.csv', index=False)

print("Merge completed")

Merge completed, results saved to IMD_65plus_merged.csv.csv
